<a href="https://colab.research.google.com/github/git4mamta/1_B-cell-Epitope-Prediction/blob/main/1_BIM2024_B_cell_Epitope_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install biopython
!pip install biopython --upgrade

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
!pip install biopython pandas
!pip install biopython pandas --upgrade

In [18]:
amino_acid_properties = {
    'A': {'hydropathy': 1.8, 'charge': 0, 'polarity': 0.16, 'flexibility': 0.357},
    'R': {'hydropathy': -4.5, 'charge': 1, 'polarity': 0.16, 'flexibility': 0.529},
    'N': {'hydropathy': -3.5, 'charge': 0, 'polarity': 0.16, 'flexibility': 0.463},
    'D': {'hydropathy': -3.5, 'charge': -1, 'polarity': 0.16, 'flexibility': 0.511},
    'C': {'hydropathy': 2.5, 'charge': 0, 'polarity': 0.16, 'flexibility': 0.346},
    'Q': {'hydropathy': -3.5, 'charge': 0, 'polarity': 0.16, 'flexibility': 0.493},
    'E': {'hydropathy': -3.5, 'charge': -1, 'polarity': 0.16, 'flexibility': 0.497},
    'G': {'hydropathy': -0.4, 'charge': 0, 'polarity': 0.16, 'flexibility': 0.544},
    'H': {'hydropathy': -3.2, 'charge': 0, 'polarity': 0.16, 'flexibility': 0.323},
    'I': {'hydropathy': 4.5, 'charge': 0, 'polarity': 0.16, 'flexibility': 0.205},
    'L': {'hydropathy': 3.8, 'charge': 0, 'polarity': 0.16, 'flexibility': 0.212},
    'K': {'hydropathy': -3.9, 'charge': 1, 'polarity': 0.16, 'flexibility': 0.466},
    'M': {'hydropathy': 1.9, 'charge': 0, 'polarity': 0.16, 'flexibility': 0.319},
    'F': {'hydropathy': 2.8, 'charge': 0, 'polarity': 0.16, 'flexibility': 0.173},
    'P': {'hydropathy': -1.6, 'charge': 0, 'polarity': 0.16, 'flexibility': 0.509},
    'S': {'hydropathy': -0.8, 'charge': 0, 'polarity': 0.16, 'flexibility': 0.447},
    'T': {'hydropathy': -0.7, 'charge': 0, 'polarity': 0.16, 'flexibility': 0.389},
    'V': {'hydropathy': 4.2, 'charge': 0, 'polarity': 0.16, 'flexibility': 0.218},
    'W': {'hydropathy': -0.9, 'charge': 0, 'polarity': 0.16, 'flexibility': 0.204},
    'Y': {'hydropathy': -1.3, 'charge': 0, 'polarity': 0.16, 'flexibility': 0.298}
}

In [19]:
from Bio.SeqUtils import ProtParamData

solv_accessibility = {
    'A': 106.0,
    'C': 135.0,
    'D': 163.0,
    'E': 194.0,
    'F': 197.0,
    'G': 84.0,
    'H': 184.0,
    'I': 169.0,
    'K': 205.0,
    'L': 164.0,
    'M': 188.0,
    'N': 157.0,
    'P': 136.0,
    'Q': 198.0,
    'R': 248.0,
    'S': 130.0,
    'T': 142.0,
    'V': 142.0,
    'W': 227.0,
    'Y': 222.0
}

ProtParamData.solv_accessibility = solv_accessibility

In [20]:
solv_accessibility

{'A': 106.0,
 'C': 135.0,
 'D': 163.0,
 'E': 194.0,
 'F': 197.0,
 'G': 84.0,
 'H': 184.0,
 'I': 169.0,
 'K': 205.0,
 'L': 164.0,
 'M': 188.0,
 'N': 157.0,
 'P': 136.0,
 'Q': 198.0,
 'R': 248.0,
 'S': 130.0,
 'T': 142.0,
 'V': 142.0,
 'W': 227.0,
 'Y': 222.0}

In [16]:
import pandas as pd
#from Bio.SeqUtils.PrimportotParam import ProteinAnalysis
import numpy as np
import io
import math
import joblib
from joblib import load


from Bio.SeqUtils.ProtParam import ProteinAnalysis

from Bio.SeqUtils import ProtParamData
from Bio.SeqUtils import ProtParam
from Bio.SeqUtils import MeltingTemp as mt
from Bio.SeqUtils.ProtParamData import kd
from Bio.SeqUtils.MeltingTemp import Tm_NN
import Bio.SeqUtils.ProtParam as Bio
from Bio.Seq import Seq
import Bio.SeqUtils.ProtParamData as Bio
from Bio.PDB.Polypeptide import is_aa
from Bio.PDB import PDBParser, PPBuilder, Selection
from Bio.PDB.vectors import calc_dihedral
from Bio.PDB import PDBParser, PPBuilder, Polypeptide
import warnings
from Bio.PDB import*
#from Bio.PDB.polypeptide import three_to_one
from Bio.SeqUtils import seq1
from Bio import SeqIO
import Bio.PDB
from Bio.PDB.Polypeptide import PPBuilder
from Bio.Seq import Seq
#from Bio.Alphabet import generic_protein
#from Bio.SeqUtils.ProtParam import gravy


df = pd.read_excel('/content/drive/My Drive/Colab Notebooks/Linear_B-cell.xlsx')
absorbance = 1.5
path_length = 1
concentration = 1e-6

valid_aas = set("ACDEFGHIKLMNPQRSTVWY")  # standard amino acids
df['Sequence'] = df['Sequence'].apply(lambda s: ''.join([aa for aa in s if aa in valid_aas]))

def calculate_hydrophilicity(sequence):
    protein = ProteinAnalysis(sequence)
    gravy = protein.gravy()
    return abs(gravy)

def calculate_surface_accessibility(sequence):
    protein = ProteinAnalysis(sequence)
    accessibility = protein.flexibility()
    surface_accessibility = sum([accessibility[i] for i, aa in enumerate(sequence) if aa in ProtParamData.solv_accessibility and ProtParamData.solv_accessibility[aa] > 0.05 and i < len(accessibility)]) / len(sequence)
    return surface_accessibility

def calculate_beta_turn(sequence):
    protein = ProteinAnalysis(sequence)
    beta_turn = protein.secondary_structure_fraction()[2]
    return beta_turn

def calculate_exposed_surface(sequence):
    protein = ProteinAnalysis(sequence)
    exposed_surface = sum([1 for i, aa in enumerate(sequence) if ProtParamData.solv_accessibility[aa] > 0]) / len(sequence)
    return exposed_surface

def calculate_polarity(sequence):
    protein = ProtParam.ProteinAnalysis(sequence)
    amino_acids = protein.count_amino_acids()
    polar_aa = ['R', 'N', 'D', 'Q', 'E', 'H', 'K', 'S', 'T', 'Y']
    polar_count = sum(amino_acids[aa] for aa in polar_aa)
    average_polarity = polar_count / len(sequence)
    return average_polarity

def calculate_antigenic(sequence):
    protein = ProteinAnalysis(sequence)
    antigenic = [amino_acid_properties[aa]['flexibility'] for aa in sequence]
    average_antigenic = sum(antigenic) / len(sequence)
    return average_antigenic

def calculate_molecular_weight(protein_sequence):
    protein = ProteinAnalysis(protein_sequence)
    mw = protein.molecular_weight()
    return mw

def calculate_hydrophobicity(sequence):
    protein = ProtParam.ProteinAnalysis(sequence)
    hydrophobicity = protein.protein_scale(window=9, param_dict=kd, edge=1)
    return hydrophobicity

def calculate_immunogenicity(sequence):
    antigenicity_scale = {
        'A': 0.8, 'R': 1.0, 'N': 0.8, 'D': 0.9, 'C': 0.8, 'Q': 0.9,
        'E': 1.0, 'G': 0.8, 'H': 0.9, 'I': 0.8, 'L': 0.8, 'K': 1.0,
        'M': 0.9, 'F': 0.8, 'P': 0.8, 'S': 0.8, 'T': 0.8, 'W': 0.8,
        'Y': 0.8, 'V': 0.8
    }
    immunogenicity = sum(antigenicity_scale.get(aa, 0) for aa in sequence) / len(sequence)
    return immunogenicity

def calculate_sequence_length(sequence):
    return len(sequence)

def calculate_electrostatic_interaction(sequence):
    # Calculate net charge of the protein sequence using ProteinAnalysis class
    protein = ProteinAnalysis(sequence)
    net_charge = protein.charge_at_pH(7.4)
    return net_charge

def calculate_flexibility(sequence):
    protein = ProteinAnalysis(sequence)
    flexibility = protein.flexibility()
    return sum(flexibility) / len(sequence)

def calculate_pI(sequence):
     protein_analysis = ProteinAnalysis(sequence)

     pKa_values = protein_analysis.isoelectric_point()

     if isinstance(pKa_values, float):
        n = 1
        pI = pKa_values
     else:
        n = len(pKa_values)
        pI = sum(pKa_values) / n

     return pI

def calculate_half_life(sequence):
    F = 0.7
    t = 10

    num_cleavable_bonds = sequence.count('D') + sequence.count('E') + sequence.count('R') + sequence.count('K')
    length = len(sequence)
    num_bonds_per_unit_length = num_cleavable_bonds / length

    if num_bonds_per_unit_length == 0:
        return None

    kdeg = (1 - F) / (t * F * num_bonds_per_unit_length)

    ln2 = math.log(2)
    half_life = 1 / (kdeg * ln2)

    return half_life


def calculate_aliphatic_index(sequence):

    rel_vol = {'A': 1.00, 'V': 2.90, 'I': 3.10, 'L': 4.90}


    num_a = sequence.count('A')
    num_v = sequence.count('V')
    num_i = sequence.count('I')
    num_l = sequence.count('L')


    sum_rel_vol = num_a * rel_vol['A'] + num_v * rel_vol['V'] + num_i * rel_vol['I'] + num_l * rel_vol['L']


    n = len(sequence)
    aliphatic_index = sum_rel_vol / n

    return aliphatic_index

def calculate_molecular_extinction_coefficient(sequence, absorbance, path_length, concentration):


    n = len(sequence)
    molecular_extinction_coefficient = absorbance / (path_length * concentration * n)

    return molecular_extinction_coefficient


def calculate_aromaticity(sequence):

    if len(sequence) % 2 != 0:
        return False

    num_pi_electrons = 0
    for atom in sequence:
        if atom in ["C", "N"]:
            num_pi_electrons += 1

    # the Hückel rule
    if num_pi_electrons % 4 == 2:
        return True
    else:
        return False

def calculate_virulence(sequence):
    virulent_amino_acids = ['G', 'K', 'R', 'F', 'Y']

    virulent_count = 0
    for aa in sequence:
        if aa in virulent_amino_acids:
            virulent_count += 1

    virulence_score = virulent_count / len(sequence)

    return virulence_score

def calculate_instability_index(sequence):
    unstable_dipeptides = ['GA', 'GS', 'GT', 'AV', 'VL', 'VA', 'IA', 'PA', 'PS', 'NP']

    unstable_count = 0
    for i in range(len(sequence) - 1):
        dipeptide = sequence[i:i+2]
        if dipeptide in unstable_dipeptides:
            unstable_count += 1

    instability_index = 10 * unstable_count / len(sequence)

    return instability_index

def calculate_adhesin_probability(sequence):
    adhesin_motifs = ['RGD', 'RGDS', 'KGD', 'NPLY', 'FNxGNxD', 'GGxxG', 'GxxG', 'DxxD']

    adhesin_count = 0
    for motif in adhesin_motifs:
        if motif in sequence:
            adhesin_count += 1

    adhesin_probability = adhesin_count / len(adhesin_motifs)

    return adhesin_probability

def calculate_df(sequence):

    def_matrix = {
        'A': 0.326,
        'C': 0.346,
        'D': 0.572,
        'E': 0.511,
        'F': 0.314,
        'G': 0.544,
        'H': 0.323,
        'I': 0.308,
        'K': 0.525,
        'L': 0.310,
        'M': 0.324,
        'N': 0.454,
        'P': 0.511,
        'Q': 0.497,
        'R': 0.548,
        'S': 0.509,
        'T': 0.444,
        'V': 0.308,
        'W': 0.305,
        'Y': 0.420
    }

    def_energy = [def_matrix[aa] for aa in sequence]

    df = sum(def_energy) / len(sequence)

    return df





'''def predict_allergenicity(sequence):
    model = joblib.load('allergenicity_model.joblib')

    # Convert the protein sequence to a feature vector
    # (e.g., using a k-mer or amino acid composition encoding)
    feature_vector = convert_sequence_to_features(sequence)

    # Make a prediction using the machine learning model
    prediction = model.predict(np.array([feature_vector]))

    # Convert the prediction to a boolean value
    if prediction == 1:
        return True
    else:
        return False
        '''





'''def calculate_dihedral_angles(fasta_seq):
    # Load sequence into BioPython structure
    parser = Bio.PDB.PDBParser()
    seq_id = "SEQ"
    sequence = Bio.PDB.Polypeptide.Polypeptide(fasta_seq, id=seq_id)

    # Calculate dihedral angles
    ppb = PPBuilder()
    structure = parser.get_structure(seq_id, sequence)
    model = structure[0]
    pp = ppb.build_peptides(model)
    angles = []
    for chain in pp:
        dihedral_list = Bio.PDB.Polypeptide.calc_dihedrals(chain)
        angles.append(dihedral_list)

    return angles'''


'''def calculate_stability(sequence):
    """
    Calculates the stability of a protein sequence based on its melting temperature
    and grand average of hydropathy (GRAVY) score.
    """
    # Calculate the melting temperature using the nearest-neighbor method
    # with the same parameters as before
    seq = Seq(sequence)
    try:
        tm = Tm_NN(seq, Na=50, Mg=1.5, dNTPs=0.8, saltcorr=7)
    except ZeroDivisionError:
        return np.nan

    # Calculate the grand average of hydropathy (GRAVY) score
    gravy = Bio.gravy(sequence)

    # Calculate the stability score as the product of the melting temperature
    # and the absolute value of the GRAVY score
    stability = abs(tm * gravy)

    return stability'''



df['Hydrophilicity'] = df['Sequence'].apply(calculate_hydrophilicity)
df['Surface Accessibility'] = df['Sequence'].apply(calculate_surface_accessibility)
df['Beta-turn'] = df['Sequence'].apply(calculate_beta_turn)
df['Exposed Surface'] = df['Sequence'].apply(calculate_exposed_surface)
df['Polarity'] =df['Sequence'].apply(calculate_polarity)
df['Antigenic'] =df['Sequence'].apply(calculate_antigenic)
df['Molecular Weight'] = df['Sequence'].apply(calculate_molecular_weight)
df['Hydrophobicity'] = df['Sequence'].apply(calculate_hydrophobicity)
df['Immunogenicity'] = df['Sequence'].apply(calculate_immunogenicity)
df['Sequence Length'] = df['Sequence'].apply(calculate_sequence_length)
df['Electrostatic Interaction'] = df['Sequence'].apply(calculate_electrostatic_interaction)
df['Instability index'] = df['Sequence'].apply(calculate_instability_index)
#df['Stability'] = df['Sequence'].apply(calculate_stability)
df['Flexibility'] = df['Sequence'].apply(calculate_flexibility)
df['pI'] = df['Sequence'].apply(calculate_pI)
df['Estimated Half Life'] = df['Sequence'].apply(calculate_half_life)
df['Aliphatic Index'] = df['Sequence'].apply(calculate_aliphatic_index)
df['Aromacity'] = df['Sequence'].apply(calculate_aromaticity)
df['Virulance'] = df['Sequence'].apply(calculate_virulence)
df['Deformability'] = df['Sequence'].apply(calculate_df)
df['Adhesin probability'] = df['Sequence'].apply(calculate_adhesin_probability)
#df['Allergenicity'] = df['Sequence'].apply(predict_allergenicity)
df['Molecular Extinction Coefficient'] = df['Sequence'].apply(calculate_molecular_extinction_coefficient, args=(absorbance, path_length, concentration))
'''for index, row in df.iterrows():
    # Get the protein sequence
    sequence = row['Sequence']

    # Create a FASTA-formatted string
    fasta_str = '>{}\n{}'.format(index, sequence)

    # Pass the FASTA-formatted sequence to a function
    #your_function(fasta_str)
    angles = calculate_dihedral_angles(fasta_str)
    print(angles)'''

df.to_excel('b_cell_epitopes_with_features.xlsx', index=False)